In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.pairwise import cosine_similarity
import random
import torch.nn.functional as F

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed) #for CPU
    torch.cuda.manual_seed(seed) #for GPU
    torch.backends.cudnn.deterministic = True

# Two-tower model architecture
class TwoTowerModel(nn.Module):
    def __init__(self, user_feature_dim, item_feature_dim, embedding_dim):
        super(TwoTowerModel, self).__init__()
        self.user_embedding_layer = nn.Linear(user_feature_dim, embedding_dim)
        self.item_embedding_layer = nn.Linear(item_feature_dim, embedding_dim)
        self.batch_norm = nn.BatchNorm1d(embedding_dim)
        
    def forward(self, user_features, item_features):
        user_embeds = self.user_embedding_layer(user_features)
        user_embeds = self.batch_norm(user_embeds)
       
        item_embeds = self.item_embedding_layer(item_features)
        item_embeds = self.batch_norm(item_embeds)
        return user_embeds, item_embeds

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, user_features, item_features, targets):
        self.user_features = user_features
        self.item_features = item_features
        self.targets = targets
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        return self.user_features[idx], self.item_features[idx], self.targets[idx]

In [3]:
# Hyperparameters
user_feature_dim = 4 # user_id, age, F/M, country
item_feature_dim = 258 # item_id, item_type, item_features
embedding_dim = 100
batch_size = 32
learning_rate = 0.001
num_epochs = 50
seed_everything(seed=666)

In [4]:
# Sample user features and item features (replace with your actual data)
user_features = torch.randn(1000, user_feature_dim)  # Assuming 1000 users with user_feature_dim-dimensional features
item_features = torch.randn(1000, item_feature_dim)  # Assuming 1000 items with item_feature_dim-dimensional features

# Sample time_spent_on_item as target (replace with your actual data)
# Assuming 100 items with a single time_spent_on_item value each (random between 1 and 60 minutes)
time_spent_on_item =  [random.randint(1, 60) for _ in range(1000)]
min_val = min(time_spent_on_item)
max_val = max(time_spent_on_item)
time_spent_on_item_normalized = [(val - min_val) / (max_val - min_val) for val in time_spent_on_item]
time_spent_on_item_normalized = torch.tensor(time_spent_on_item_normalized, dtype=torch.float32)

# time_spent_on_item_converted = [((x - min_val) / (max_val - min_val) - 0.5) * 2 for x in time_spent_on_item]
# time_spent_on_item_converted = torch.tensor(time_spent_on_item_converted, dtype=torch.float32)
# print(time_spent_on_item[:5], time_spent_on_item_converted[:5])

In [5]:
# Create the two-tower model
model = TwoTowerModel(user_feature_dim, item_feature_dim, embedding_dim)

# Create the custom dataset and data loader
dataset = CustomDataset(user_features, item_features, time_spent_on_item_normalized)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [6]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0.0
    for user_feats, item_feats, targets in data_loader:
        model.train()
        optimizer.zero_grad()
        
        user_embeds, item_embeds = model(user_feats, item_feats)
        cos_sim = nn.functional.cosine_similarity(user_embeds, item_embeds)
        loss = criterion(cos_sim.unsqueeze(1), targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(data_loader)}")

/Users/libai/anaconda3/envs/RR/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/libai/anaconda3/envs/RR/lib/python3.9/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/50, Loss: 0.34596234234049916
Epoch 2/50, Loss: 0.2722171812783927
Epoch 3/50, Loss: 0.24700147286057472
Epoch 4/50, Loss: 0.22438882989808917
Epoch 5/50, Loss: 0.21077052131295204
Epoch 6/50, Loss: 0.19836670230142772
Epoch 7/50, Loss: 0.18632018263451755
Epoch 8/50, Loss: 0.17391128418967128
Epoch 9/50, Loss: 0.17174197966232896
Epoch 10/50, Loss: 0.15881346142850816
Epoch 11/50, Loss: 0.1563244671560824
Epoch 12/50, Loss: 0.15171588864177465
Epoch 13/50, Loss: 0.1391624074894935
Epoch 14/50, Loss: 0.137031233869493
Epoch 15/50, Loss: 0.1351512351538986
Epoch 16/50, Loss: 0.12719742092303932
Epoch 17/50, Loss: 0.12180925044231117
Epoch 18/50, Loss: 0.12113561015576124
Epoch 19/50, Loss: 0.1166975584346801
Epoch 20/50, Loss: 0.11368030915036798
Epoch 21/50, Loss: 0.11073828092776239
Epoch 22/50, Loss: 0.10603262367658317
Epoch 23/50, Loss: 0.1043703742325306
Epoch 24/50, Loss: 0.10124758118763566
Epoch 25/50, Loss: 0.09947195276618004
Epoch 26/50, Loss: 0.1015332406386733
Epoc